In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.stats import truncnorm

# Custom
from matplotlib import cm
from scipy.optimize import minimize
from scipy.special import logsumexp
from sklearn.mixture import GaussianMixture

from density import get_projected_coords, get_u_v, get_uniform_idx, GaussianNoodle2D
from likelihood import z_to_a, a_to_z, StreamDensityModel

In [ ]:
rnd = np.random.RandomState(22)

h = 1.
true_K = 3
true_nodes = np.stack((np.linspace(0, 10, true_K),
                       np.zeros(true_K))).T
true_s = rnd.uniform(0.2, 0.5, size=true_K)

true_z = rnd.uniform(0.8, 0.95, size=true_K-1)
true_m = np.zeros(true_K)
true_a = z_to_a(true_z)
assert np.isclose(true_a.sum(), 1.)

In [ ]:
# Sample from truth:
C = np.zeros((true_K, 2, 2))
C[:, 0, 0] = h**2
C[:, 1, 1] = true_s**2

P = np.zeros_like(C)
for k in range(true_K):
    P[k] = np.linalg.cholesky(np.linalg.inv(C[k]))

gmm = GaussianMixture(n_components=true_K, covariance_type='full')
gmm.fit(np.random.random(size=(true_K+1, 2)))
gmm.weights_ = true_a
gmm.covariances_ = C
gmm.precisions_cholesky_ = P
gmm.means_ = true_nodes

# now add a uniform background
n_samples_total = 10000
true_f = 0.5
n_stream = int(true_f * n_samples_total)
gmm_X = gmm.sample(n_samples=n_stream)[0]

Background points:

In [ ]:
window_bounds = [[-5, 15], [-5, 5]]
n_bg = n_samples_total - n_stream

true_bg_mu = 20
true_bg_sigma = 5
myclip_a, myclip_b = window_bounds[0]
a, b = (myclip_a - true_bg_mu) / true_bg_sigma, (myclip_b - true_bg_mu) / true_bg_sigma
bg_x1 = truncnorm.rvs(a, b, loc=true_bg_mu, scale=true_bg_sigma, size=n_bg)
bg_x2 = np.random.uniform(window_bounds[1][0], window_bounds[1][1], 
                          size=n_bg)
bg_X = np.stack((bg_x1, bg_x2)).T

X = np.vstack((gmm_X, bg_X))

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(X[:, 0], X[:, 1], marker='.', ls='none')
plt.scatter(true_nodes[:, 0], true_nodes[:, 1], zorder=10, color='r')
# plt.scatter(nodes[:, 0], nodes[:, 1], zorder=10, color='k')
ax = plt.gca()
ax.set_aspect('equal')
ax.set_xlim(-5, 15)
ax.set_ylim(-5, 5)

Take derivatives of bg model:

In [ ]:
x1_grid = np.linspace(window_bounds[0][0], window_bounds[0][1], 1024)
x2_grid = np.zeros(len(x1_grid))
X_grid = np.stack((x1_grid, x2_grid)).T

bg_p0 = {'bg_c1': 1e-1, 
         'bg_c2': 0, 
         'bg_c3': 1.,
         'bg_x0': -5.}

bg_ll = ln_bg(bg_p0, X_grid)

plt.plot(x1_grid, np.exp(bg_ll))
plt.plot(x1_grid, truncnorm.pdf(x1_grid, a, b, loc=true_bg_mu, scale=true_bg_sigma) / 10.)

In [ ]:
def ln_d_bg_dp(p, X):
    a, b = window_bounds[0]
    x, x2 = X.T
    N = len(x)
    
    c1 = p['bg_c1']
    c2 = p['bg_c2']
    c3 = p['bg_c3']
    x0 = p['bg_x0']
    
    
    derivs = dict()
    signs = dict()
    
    derivs['bg_c1'] = ((6*(2*a**2*(c3 + c2*(x - x0)) + 2*b**2*(c3 + c2*(x - x0)) - 
                          6*x*(c3*(x - 2*x0) + c2*x0*(-x + x0)) - 3*b*(2*c3*x0 + c2*(x - x0)*(x + x0)) + 
                          a*(-3*c2*x**2 - 6*c3*x0 + 3*c2*x0**2 + 2*b*(c3 + c2*x - c2*x0)))) /
                        ((a - b)*(2*a**2*c1 + 2*a*b*c1 + 2*b**2*c1 + 3*a*c2 + 3*b*c2 + 6*c3 - 
                                  6*((a + b)*c1 + c2)*x0 + 6*c1*x0**2)**2))
    
    derivs['bg_c2'] = ((6*(2*a**2*c1*(-x + x0) + 2*b**2*c1*(-x + x0) - 6*x*(c3 + c1*(x - x0)*x0) + 
   3*b*(c3 + c1*(x - x0)*(x + x0)) + a*(3*c3 + c1*(x - x0)*(-2*b + 3*(x + x0)))))/
 ((a - b)*(2*a**2*c1 + 2*a*b*c1 + 2*b**2*c1 + 3*a*c2 + 3*b*c2 + 6*c3 - 
    6*((a + b)*c1 + c2)*x0 + 6*c1*x0**2)**2))
    
    derivs['bg_c3'] = ((-6*(2*a**2*c1 + 2*b**2*c1 + 3*b*c2 + a*(2*b*c1 + 3*c2) - 6*x*(c2 + c1*x)) + 
  36*c1*(a + b - 2*x)*x0)/((a - b)*(2*a**2*c1 + 2*a*b*c1 + 2*b**2*c1 + 3*a*c2 + 3*b*c2 + 
    6*c3 - 6*((a + b)*c1 + c2)*x0 + 6*c1*x0**2)**2))
    
    derivs['bg_x0'] = ((6*(6*(c2 + c1*(a + b - 2*x0))*(c3 + (c2 + c1*(x - x0))*(x - x0)) - 
   (c2 + 2*c1*(x - x0))*(2*a**2*c1 + 2*a*b*c1 + 2*b**2*c1 + 3*a*c2 + 3*b*c2 + 6*c3 - 
     6*((a + b)*c1 + c2)*x0 + 6*c1*x0**2)))/
 ((-a + b)*(2*a**2*c1 + 2*a*b*c1 + 2*b**2*c1 + 3*a*c2 + 3*b*c2 + 6*c3 - 
    6*((a + b)*c1 + c2)*x0 + 6*c1*x0**2)**2))
    
    # because it's multiplied in the likelihood
    ln_px2 = -np.log(window_bounds[1][1] - window_bounds[1][0])
    
    for name in ['bg_c1', 'bg_c2', 'bg_c3', 'bg_x0']:
        signs[name] = np.sign(derivs[name])
        derivs[name] = np.log(np.abs(derivs[name])) + ln_px2
    
    return derivs, signs 

In [ ]:
# ln_d_bg_dp(bg_p0, X_grid)

In [ ]:
true_dens_model = GaussianNoodle2D(X, poly=np.poly1d([0.]))
track = true_dens_model.get_dense_poly_track(size=10000)
nodes = true_dens_model.set_nodes(track=track, nodes=true_nodes)

In [ ]:
bg_params = {'bg_c1': (1,),
             'bg_c2': (1,),
             'bg_c3': (1,),
             'bg_x0': (1,)}
model = StreamDensityModel(X, true_dens_model, h=h, 
                           bg_ln_likelihood=ln_bg, 
                           d_bg_ln_likelihood_dp=ln_d_bg_dp,
                           bg_params=bg_params,
                           m_prior_sigma=0.1,
                           frozen=dict(bg_c2=0., 
                                       bg_c3=0.))

In [ ]:
p0 = {'ln_s': np.log(true_s),
      'ln_z': np.log(true_z),
      'm': true_m,
      'f': true_f}
p0.update(bg_p0)

In [ ]:
model.d_ln_likelihood_dp(p0)

In [ ]:
def func_helper(x):
    p = model.unpack_pars(x)
    return -model.ln_likelihood(p).sum()

def num_deriv_helper(x):
    eps = 1e-10
    num_derivs = []
    for k in range(x.size):
        x1 = x.copy()
        x2 = x1.copy()
        x2[k] = x2[k] + eps
        _d = (func_helper(x2) - func_helper(x1)) / eps
        num_derivs.append(_d)
    return np.array(num_derivs)

def deriv_helper(x):
    p = model.unpack_pars(x)
    derivs = model.d_ln_likelihood_dp(p)
    return -np.concatenate(list(derivs.values()))

In [ ]:
x0 = model.pack_pars(p0, fill_frozen=False)
true_x = x0

In [ ]:
x0

In [ ]:
p0

In [ ]:
func_helper(x0)

In [ ]:
deriv_helper(x0)

In [ ]:
num_deriv_helper(x0)

In [ ]:
jj = model._params_sorted_unfrozen.index('f')
vals = np.linspace(0.1, 1-1e-3, 128)
# vals = np.linspace(np.log(0.1), np.log(5), 128)
lls = []
for val in vals:
    xx = x0.copy()
    xx[jj] = val
    lls.append(-func_helper(xx))
lls = np.array(lls)
plt.plot(vals, np.exp(lls - lls.max()))
plt.axvline(true_x[jj])

In [ ]:
K = true_K

In [ ]:
# min_bounds = [[None,None]]*3 + [[-5, 15]]
# min_bounds = [[0,1e-2], [-1, 1], [-10, 10], [-5, 15]]
# min_bounds = [[0,1e-2], [-1, 1], [-10, 10], [-5, 15]]
min_bounds = [[None,None]] + [[-15, 15]]
min_bounds = min_bounds +  [[1e-5, 1-1e-5]] + [[-8, 4]]*K + [[-8, -1e-9]]*(K-1) + [[-5, 5]]*K

res = minimize(func_helper, jac=deriv_helper, 
               x0=x0, method='L-BFGS-B', 
               bounds=min_bounds,
               options=dict(ftol=1e-13))

In [ ]:
res
assert res.success

In [ ]:
bins = (np.arange(-10, 15+1e-3, 0.05),
        np.arange(-5, 5+1e-3, 0.05))
x, y = np.meshgrid(*bins)
X_grid = np.stack((x.ravel(), y.ravel())).T

In [ ]:
best_p = model.unpack_pars(res.x)
ln_dens = model.ln_likelihood(best_p, X_grid).reshape(x.shape)

In [ ]:
# derp = np.exp(ln_dens - ln_dens.max())
# plt.pcolormesh(x, y, derp)

plt.pcolormesh(x, y, ln_dens)